https://learn.deeplearning.ai/langchain-chat-with-your-data

In [1]:
import openai
import config
openai.api_key = config.OPENAI_KEY


#### Loading and Chunking Input Documents

In [2]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("docs/891100-Weihnachtsbrief.pdf")
pages = loader.load()

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

chunk_size = 400
chunk_overlap = 200

r_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size,
    chunk_overlap = chunk_overlap,
    separators= ["\n\n", "\n", "(?<=\. )", " ", ]
)

docs = r_splitter.split_documents(pages)
len(docs)

101

#### Embedding and VectorDB

In [5]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(openai_api_key=config.OPENAI_KEY)
persist_directory = "chroma/"

In [6]:
# new db from documents

vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embedding,
    persist_directory=persist_directory
)

vectordb.persist()

#### Retrieving relevant chunks

In [7]:
# reopen existing db

vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [23]:
question = "Was haben die beiden im Norden erlebt?"
docs = vectordb.similarity_search(question, k=3)

In [9]:
docs = vectordb.max_marginal_relevance_search(question,k=2, fetch_k=3)

In [10]:
print(docs)

[Document(page_content='der Universitat in Zürich, wo er Volkswirtschaft mit Fleiss und Interesse studiert. \nEr ist jetzt daran, sich auf sein erstes Vordiplom vorzubereiten. Im Sommer ver\xad\ndiente er sich sein Geld an einer Amtsstelle der Stadt Zürich für das Flüchtlings\xad\nwesen -ein Kapitel, das die ganze Schweiz sehr beschaftigt. Mir hat Jürg ganz', metadata={'page': 1, 'source': 'docs/891100-Weihnachtsbrief.pdf'}), Document(page_content='in einer gut geführten Meditationsgruppe. \nUnsere Grosskinder: \nDie zwei Spindler-Buben in Greifensee sind noch nicht aus dem Streckalter heraus\xad\ngekommen. Jürg ist jetzt der grosste Spindler in der Schweiz. Es gefallt ihm an \nder Universitat in Zürich, wo er Volkswirtschaft mit Fleiss und Interesse studiert. \nEr ist jetzt daran, sich auf sein erstes Vordiplom vorzubereiten. Im Sommer ver\xad', metadata={'page': 1, 'source': 'docs/891100-Weihnachtsbrief.pdf'})]


#### Using GPT for question answering

In [11]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(openai_api_key=config.OPENAI_KEY, model_name="gpt-4", temperature=0)

In [24]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)
result = qa_chain({"query": question})
result["result"]

'Der Text gibt nicht explizit an, wer in den Norden gereist ist. Es wird nur erwähnt, dass "wir" von Baden nach Hamburg und dann nach Norwegen gereist sind. Es ist daher anzunehmen, dass der Autor oder die Autorin und mindestens eine weitere Person in den Norden gereist sind.'

#### Using GPT for task generation

In [11]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Erzeuge eine Single-Choice-Aufgabe mit 4 Antwortoptionen, wovon eine richtig ist. Die Aufgabe soll folgendes Wissen prüfen: 
{context}
Single-Choice-Aufgabe:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [12]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [15]:
question = "Baltimore"
result = qa_chain({"query": question})
result["source_documents"]

[Document(page_content='zu synthetisieren. Baltimore  erhielt dafür 1975 (im Alter von erst 37 Jahren) den Nobelpreis  für Medi-\nzin. Ein weiterer Meilenstein in der Genetik war 1967 die Entdeckung  von Enzymen, die die DNA an \nspezifischen Stellen  schneiden können (Restriktion senzyme; Nobelpreis für Medizin  1978 an Werner', metadata={'page': 3, 'source': 'exampletext.pdf'}),
 Document(page_content='zu synthetisieren. Baltimore  erhielt dafür 1975 (im Alter von erst 37 Jahren) den Nobelpreis  für Medi-\nzin. Ein weiterer Meilenstein in der Genetik war 1967 die Entdeckung  von Enzymen, die die DNA an \nspezifischen Stellen  schneiden können (Restriktion senzyme; Nobelpreis für Medizin  1978 an Werner', metadata={'page': 3, 'source': 'exampletext.pdf'}),
 Document(page_content='die Richtung de s Informationsflusses (DNA → RNA → Protein) als »Einbahnstraße«. Dieses »zent-\nrale Dogma der Genetik« wurde 1970 umgestoßen,  als David Baltimore über das Enzym Reverse  \nTranskriptase  (au

In [14]:
result["result"]

'Für welche Entdeckung erhielt David Baltimore 1975 den Nobelpreis für Medizin?\n\nA) Für die Entdeckung der Restriktionsenzyme\nB) Für die Entdeckung der Reverse Transkriptase\nC) Für die Entdeckung der DNA\nD) Für die Entdeckung der RNA-Tumorviren\n\nRichtige Antwort: B) Für die Entdeckung der Reverse Transkriptase'